In [1]:
from collections import Counter
import numpy as np
import tensorflow.contrib.keras as kr
import tensorflow as tf
import time
from datetime import timedelta
import os
from sklearn import metrics
from sklearn.model_selection import KFold

import moxing as mox
mox.file.shift('os', 'mox')

INFO:root:Using MoXing-v1.14.1-ddfd6c9a
INFO:root:Using OBS-Python-SDK-3.1.2


In [2]:
trainDataPath = "s3://corpus-2/dataset/corpus_5.txt"
vocabPath = "s3://corpus-text-classification1/data/glove.6B.100d.txt"
savePath = "s3://corpus-2/model/cnn2"

In [11]:
def readfile(filePath):
    """读取文件内容，返回文本和标签列表"""
    train_data = []
    with open(filePath, 'r', encoding='utf-8') as f:
        for line in f.readlines():
            word = line.split()
            label = int(word[0].split(":")[0])
            content = word[1:]
            train_data.append([content,label])

    return np.asarray(train_data)


def readCategory():
    """读取分类目录，固定id"""
    '''
    Retrieve Value
    Filter
    Compute Derived Value
    Find Extremum
    Sort
    Determine Range
    Characterize Distribution
    Find Anomalies
    Cluster
    Correlate
    '''
    categories = ['Retrieve Value', 'Filter', 'Compute Derived Value', 'Find Extremum', 'Sort', 
                  'Determine Range', 'Characterize Distribution', 'Find Anomalies', 'Cluster', 'Correlate']
    cat_to_id = dict(zip(categories, range(1,len(categories)+1)))
    id_to_cat = dict(zip(range(1,len(categories)+1), categories))
    return id_to_cat, cat_to_id


def loadGloVe(filename, emb_size=50):
    vocab = []
    embd = []
    print('Loading GloVe!')
    # vocab.append('unk') #装载不认识的词
    # embd.append([0] * emb_size) #这个emb_size可能需要指定
    file = open(filename,'r',encoding='utf-8')
    for line in file.readlines():
        row = line.strip().split(' ')
        vocab.append(row[0])
        embd.append([float(ei) for ei in row[1:]])
    file.close()
    print('Completed!')
    return vocab,embd

In [12]:
train_data = readfile(trainDataPath)

In [13]:
len(train_data[0]),len(train_data)

(2, 14035)

In [22]:
np.random.shuffle(train_data)
len(train_data[:,0])

14035

In [24]:
train_data[0]

array([list(['can', 'you', 'find', 'anything', 'unusual', 'about', 'the', 'average', 'yearly', 'rainfall', 'of', 'each', 'chinese', 'province', 'this', 'year', '?']),
       8], dtype=object)

In [15]:
seq_length = 0
for content in train_data[:,0]:
    if seq_length < len(content):
        seq_length = len(content)   # seq_length = 41

In [16]:
seq_length

41

In [17]:
vocab, embd = loadGloVe(vocabPath, 100)
vocab_size = len(vocab)
embedding_dim = len(embd[0])
embedding = np.asarray(embd)

Loading GloVe!
Completed!


In [18]:
word_to_id = dict(zip(vocab, range(vocab_size)))

In [19]:
len(embedding),embedding_dim,vocab_size

(400000, 100, 400000)

In [20]:
def process_file(contents, labels, word_to_id, num_classes, pad_max_length):
    """
    将文件转换为id表示,并且将每个单独的样本长度固定为pad_max_lengtn
    """
    # contents, labels = readfile(filePath)
    data_id, label_id = [], []
    # 将文本内容转换为对应的id形式
    for i in range(len(contents)):
        data_id.append([word_to_id[x] for x in contents[i] if x in word_to_id])
        label_id.append(labels[i] - 1)
    # 使用keras提供的pad_sequences来将文本pad为固定长度
    x_pad = kr.preprocessing.sequence.pad_sequences(data_id, pad_max_length)
    ''' https://blog.csdn.net/TH_NUM/article/details/80904900
    pad_sequences(sequences, maxlen=None, dtype=’int32’, padding=’pre’, truncating=’pre’, value=0.) 
        sequences：浮点数或整数构成的两层嵌套列表
        maxlen：None或整数，为序列的最大长度。大于此长度的序列将被截短，小于此长度的序列将在后部填0.
        dtype：返回的numpy array的数据类型
        padding：‘pre’或‘post’，确定当需要补0时，在序列的起始还是结尾补
        truncating：‘pre’或‘post’，确定当需要截断序列时，从起始还是结尾截断
        value：浮点数，此值将在填充时代替默认的填充值0
    '''
    y_pad = kr.utils.to_categorical(label_id, num_classes=num_classes)  # 将标签转换为one-hot表示
    ''' https://blog.csdn.net/nima1994/article/details/82468965
    to_categorical(y, num_classes=None, dtype='float32')
        将整型标签转为onehot。y为int数组，num_classes为标签类别总数，大于max(y)（标签从0开始的）。
        返回：如果num_classes=None，返回len(y) * [max(y)+1]（维度，m*n表示m行n列矩阵，下同），否则为len(y) * num_classes。
    '''
    return x_pad, y_pad


def get_time_dif(start_time):
    """获取已使用时间"""
    end_time = time.time()
    time_dif = end_time - start_time
    return timedelta(seconds=int(round(time_dif)))

In [25]:
categories = ['Retrieve Value', 'Filter', 'Compute Derived Value', 'Find Extremum', 'Sort', 
                  'Determine Range', 'Characterize Distribution', 'Find Anomalies', 'Cluster', 'Correlate']

num_classes = len(categories)

print("Loading training and validation and testing data...")
start_time = time.time()
x_train, y_train = process_file(train_data[:,0], train_data[:,1], word_to_id, num_classes, seq_length)
time_dif = get_time_dif(start_time)
print("Loading data Time usage:", time_dif)

Loading training and validation and testing data...
Loading data Time usage: 0:00:00


In [26]:
x_train[0], y_train[0]

(array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,   86,   81,  596, 1096, 3217,   59,    0,  641, 8946,
        8707,    3,  236,  327,  624,   37,   62,  188], dtype=int32),
 array([0., 0., 0., 0., 0., 0., 0., 1., 0., 0.]))

In [16]:
len(x_train)

14035

In [17]:
word_to_id["arkansas"]

4791

In [18]:
embedding[3]

array([-0.1529  , -0.24279 ,  0.89837 ,  0.16996 ,  0.53516 ,  0.48784 ,
       -0.58826 , -0.17982 , -1.3581  ,  0.42541 ,  0.15377 ,  0.24215 ,
        0.13474 ,  0.41193 ,  0.67043 , -0.56418 ,  0.42985 , -0.012183,
       -0.11677 ,  0.31781 ,  0.054177, -0.054273,  0.35516 , -0.30241 ,
        0.31434 , -0.33846 ,  0.71715 , -0.26855 , -0.15837 , -0.47467 ,
        0.051581, -0.33252 ,  0.15003 , -0.1299  , -0.54617 , -0.37843 ,
        0.64261 ,  0.82187 , -0.080006,  0.078479, -0.96976 , -0.57741 ,
        0.56491 , -0.39873 , -0.057099,  0.19743 ,  0.065706, -0.48092 ,
       -0.20125 , -0.40834 ,  0.39456 , -0.02642 , -0.11838 ,  1.012   ,
       -0.53171 , -2.7474  , -0.042981, -0.74849 ,  1.7574  ,  0.59085 ,
        0.04885 ,  0.78267 ,  0.38497 ,  0.42097 ,  0.67882 ,  0.10337 ,
        0.6328  , -0.026595,  0.58647 , -0.44332 ,  0.33057 , -0.12022 ,
       -0.55645 ,  0.073611,  0.20915 ,  0.43395 , -0.012761,  0.089874,
       -1.7991  ,  0.084808,  0.77112 ,  0.63105 , 

In [27]:
class rnn_cnn(object):
    
    def __init__(self, savePath, num_classes):
        self.savePath = savePath
        # 输入内容及对应的标签
        self.input_x = tf.placeholder(tf.int32, [None, seq_length], name='input_x')
        self.input_y = tf.placeholder(tf.float32, [None, num_classes], name='input_y')
        # dropout的损失率
        self.keep_prob = tf.placeholder(tf.float64, name='keep_prob')
        # 词向量映射;实际上此处的词向量并不是用的预训练好的词向量，而是未经任何训练直接生成了一个矩阵，将此矩阵作为词向量矩阵使用，效果也还不错。
        # 若使用训练好的词向量，或许训练此次文本分类的模型时会更快，更好。
        # embedding = tf.get_variable('embedding', [vocab_size, embedding_dim])
        embedding_inputs = tf.nn.embedding_lookup(embedding, self.input_x)

        num_filters = 256
        kernel_size = 5
        hidden_dim = 128
        learning_rate = 1e-3

        # CNN layer
        conv = tf.layers.conv1d(embedding_inputs, num_filters, kernel_size, name='conv')  # num_filters = 256 这是个啥
        ''' https://blog.csdn.net/khy19940520/article/details/89934335
        tf.layers.conv1d：一维卷积一般用于处理文本数据，常用语自然语言处理中，输入一般是文本经过embedding的二维数据。
            inputs： 输入tensor， 维度(batch_size, seq_length, embedding_dim) 是一个三维的tensor；其中，
                batch_size指每次输入的文本数量；
                seq_length指每个文本的词语数或者单字数；
                embedding_dim指每个词语或者每个字的向量长度；
                例如每次训练输入2篇文本，每篇文本有100个词，每个词的向量长度为20，那input维度即为(2, 100, 20)。
            filters：过滤器（卷积核）的数目
            kernel_size：卷积核的大小，卷积核本身应该是二维的，这里只需要指定一维，因为第二个维度即长度与词向量的长度一致，卷积核只能从上往下走，不能从左往右走，即只能按照文本中词的顺序，也是列的顺序。
        '''
        # global max pooling layer
        gmp = tf.reduce_max(conv, reduction_indices=[1], name='gmp')  # https://blog.csdn.net/lllxxq141592654/article/details/85345864

        # 全连接层，后面接dropout以及relu激活
        fc = tf.layers.dense(gmp, hidden_dim, name='fc1')  # hidden_dim：128
        ''' https://blog.csdn.net/yangfengling1023/article/details/81774580
        dense ：全连接层  inputs：输入该网络层的数据；units：输出的维度大小，改变inputs的最后一维
        '''
        fc = tf.nn.dropout(fc, self.keep_prob)
        fc = tf.nn.relu(fc)

        # 分类器
        logits = tf.layers.dense(fc, num_classes, name='fc2')
        self.y_pred_cls = tf.argmax(tf.nn.softmax(logits), 1)  # 预测类别 tf.argmax：返回每一行或每一列的最大值 1为里面（每一行），0为外面（每一列）

        # 损失函数，交叉熵
        cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=self.input_y)
        self.loss = tf.reduce_mean(cross_entropy)
        # 优化器
        self.optim = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(self.loss)

        # 准确率
        correct_pred = tf.equal(tf.argmax(self.input_y, 1), self.y_pred_cls)
        self.acc = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
        
        self.saver = tf.train.Saver()
        
    
    def train(self, x_train, y_train, split_type, fold_id, num_epochs=20, dropout_keep_prob=0.5, print_per_batch=30, batch_size=64):
        
        savePath = "%s/%s/%s/%s" % (self.savePath, split_type, fold_id, fold_id)
        # 创建session
        session = tf.Session()
        session.run(tf.global_variables_initializer())
        
        print('Training and evaluating...')
        start_time = time.time()
        total_batch = 0  # 总批次
        best_acc_train = 0.0  # 最佳验证集准确率
        last_improved = 0  # 记录上一次提升批次
        require_improvement = 500  # 如果超过1000轮未提升，提前结束训练
        flag = False

        for epoch in range(num_epochs):  # 20
            print('Epoch:', epoch + 1)
            batch_train = rnn_cnn.batch_iter(x_train, y_train, batch_size)
            for x_batch, y_batch in batch_train:
                feed_dict = {self.input_x: x_batch, self.input_y: y_batch, self.keep_prob: dropout_keep_prob}
                session.run(self.optim, feed_dict=feed_dict)  # 运行优化
                total_batch += 1

                if total_batch % print_per_batch == 0:
                    # 每多少轮次输出在训练集和验证集上的性能
                    feed_dict[self.keep_prob] = 1.0
                    loss_train, acc_train = session.run([self.loss, self.acc], feed_dict=feed_dict)
                    # loss_val, acc_val = evaluate(session, x_dev, y_dev, loss, acc)
                    if acc_train > best_acc_train:
                        # 保存最好结果
                        best_acc_train = acc_train
                        last_improved = total_batch
                        self.saver.save(sess=session, save_path=savePath)
                        improved_str = '*'
                    else:
                        improved_str = ''

                    time_dif = get_time_dif(start_time)
                    msg = 'Iter: {0:>6}, Train Loss: {1:>6.2}, Train Acc: {2:>7.2%}, Time: {3} {4}'
                    print(msg.format(total_batch, loss_train, acc_train, time_dif, improved_str))

                if total_batch - last_improved > require_improvement:
                    # 验证集正确率长期不提升，提前结束训练
                    print("No optimization for a long time, auto-stopping...")
                    flag = True
                    break  # 跳出循环
            if flag:  # 同上
                break
                
        session.close()
        return best_acc_train
        
        
    def evaluate_model(self, x_test, y_test, split_type, fold_id, categories, batch_size=64):
        
        savePath = "%s/%s/%s/%s" % (self.savePath, split_type, fold_id, fold_id)
        # 读取保存的模型
        session = tf.Session()
        self.saver.restore(sess=session, save_path=savePath)
        start_time = time.time()
        print('Testing...')
        loss_test, acc_test = self.evaluate(session, x_test, y_test, self.loss, self.acc, batch_size)
        msg = 'Test Loss: {0:>6.2}, Test Acc: {1:>7.2%}'
        print(msg.format(loss_test, acc_test))

        test_data_len = len(x_test)
        test_num_batch = int((test_data_len - 1) / batch_size) + 1

        y_test_cls = np.argmax(y_test, 1)  # 获得类别
        y_test_pred_cls = np.zeros(shape=len(x_test), dtype=np.int32)  # 保存预测结果  len(x_test) 表示有多少个文本

        for i in range(test_num_batch):  # 逐批次处理
            start_id = i * batch_size
            end_id = min((i + 1) * batch_size, test_data_len)
            feed_dict = {
                self.input_x: x_test[start_id:end_id],
                self.keep_prob: 1.0
            }
            y_test_pred_cls[start_id:end_id] = session.run(self.y_pred_cls, feed_dict=feed_dict)

        # 评估
        print("Precision, Recall and F1-Score...")
        print(metrics.classification_report(y_test_cls, y_test_pred_cls, target_names=categories))
        '''
        sklearn中的classification_report函数用于显示主要分类指标的文本报告．在报告中显示每个类的精确度，召回率，F1值等信息。
            y_true：1维数组，或标签指示器数组/稀疏矩阵，目标值。 
            y_pred：1维数组，或标签指示器数组/稀疏矩阵，分类器返回的估计值。 
            labels：array，shape = [n_labels]，报表中包含的标签索引的可选列表。 
            target_names：字符串列表，与标签匹配的可选显示名称（相同顺序）。 
            原文链接：https://blog.csdn.net/akadiao/article/details/78788864
        '''

        # 混淆矩阵
        print("Confusion Matrix...")
        cm = metrics.confusion_matrix(y_test_cls, y_test_pred_cls)
        '''
        混淆矩阵是机器学习中总结分类模型预测结果的情形分析表，以矩阵形式将数据集中的记录按照真实的类别与分类模型作出的分类判断两个标准进行汇总。
        这个名字来源于它可以非常容易的表明多个类别是否有混淆（也就是一个class被预测成另一个class）
        https://blog.csdn.net/u011734144/article/details/80277225
        '''
        print(cm)

        time_dif = get_time_dif(start_time)
        print("Time usage:", time_dif)
        session.close()
        
        return acc_test
        
        
    def predict(self, predict_sentences, word_to_id, pad_max_length, split_type, fold_id):
        """
        将文件转换为id表示,并且将每个单独的样本长度固定为pad_max_lengtn
        """
        savePath = "%s/%s/%s/%s" % (self.savePath, split_type, fold_id, fold_id)
        session = tf.Session()
        self.saver.restore(sess=session, save_path=savePath)
        
        data_id = []
        # 将文本内容转换为对应的id形式
        for i in range(len(predict_sentences)):
            data_id.append([word_to_id[x] for x in predict_sentences[i].lower().strip().split() if x in word_to_id])

        # 使用keras提供的pad_sequences来将文本pad为固定长度
        x_pad = kr.preprocessing.sequence.pad_sequences(data_id, pad_max_length)
        ''' https://blog.csdn.net/TH_NUM/article/details/80904900
        pad_sequences(sequences, maxlen=None, dtype=’int32’, padding=’pre’, truncating=’pre’, value=0.) 
            sequences：浮点数或整数构成的两层嵌套列表
            maxlen：None或整数，为序列的最大长度。大于此长度的序列将被截短，小于此长度的序列将在后部填0.
            dtype：返回的numpy array的数据类型
            padding：‘pre’或‘post’，确定当需要补0时，在序列的起始还是结尾补
            truncating：‘pre’或‘post’，确定当需要截断序列时，从起始还是结尾截断
            value：浮点数，此值将在填充时代替默认的填充值0
        '''
        feed_dict = {
            self.input_x: x_pad,
            self.keep_prob: 1.0
        }
        predict_result = session.run(self.y_pred_cls, feed_dict=feed_dict)
        predict_result = [i+1 for i in predict_result]
        session.close()
        
        return predict_result
    
    
    def evaluate(self, sess, x_pad, y_pad, loss1, acc1, batch_size):
        """评估在某一数据上的准确率和损失"""
        data_len = len(x_pad)
        batch_eval = rnn_cnn.batch_iter(x_pad, y_pad, batch_size)  
        total_loss = 0.0
        total_acc = 0.0
        # print(dropout_keep_prob)
        for x_batch1, y_batch1 in batch_eval:
            batch_len = len(x_batch1)
            feed_dict1 = {self.input_x: x_batch1, self.input_y: y_batch1, self.keep_prob: 1.0}
            lossTmp, accTmp = sess.run([loss1, acc1], feed_dict=feed_dict1)
            total_loss += lossTmp * batch_len
            total_acc += accTmp * batch_len

        return total_loss / data_len, total_acc / data_len
    
    
    def batch_iter(x_pad, y_pad, batch_size=64):  # 128
        """生成批次数据"""
        data_len = len(x_pad)
        num_batch = int((data_len - 1) / batch_size) + 1
        # np.arange()生成0到data_len的等差数列，默认等差为1；np.random.permutation()打乱生成的等差序列的顺序
        # 下面三句语句是为了将训练或测试文本的顺序打乱，因为原文本中每个分类的样本全部挨在一起，这样每个batch训练的都是同一个分类，不太好，打乱后每个batch可包含不同分类
        indices = np.random.permutation(np.arange(data_len))
        x_shuffle = x_pad[indices]
        y_shuffle = y_pad[indices]

        # 返回所有batch的数据
        for i in range(num_batch):
            start_id = i * batch_size
            end_id = min((i + 1) * batch_size, data_len)
            yield x_shuffle[start_id:end_id], y_shuffle[start_id:end_id]
            
            

In [29]:
kf = KFold(n_splits=10)
fold_id = 0
model_train_acc = []
model_test_acc = []
model = rnn_cnn(savePath, num_classes)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [30]:
split_type = "random"
for train_i, test_i in kf.split(x_train):
    fold_id += 1
    print("Fold: ", fold_id)
    model_train_acc.append(model.train(x_train[train_i], y_train[train_i],split_type,fold_id))
    model_test_acc.append(model.evaluate_model(x_train[test_i], y_train[test_i],split_type,fold_id,categories))

Fold:  1
Training and evaluating...
Epoch: 1
Iter:     30, Train Loss:    2.1, Train Acc:  29.69%, Time: 0:00:08 *
Iter:     60, Train Loss:    1.7, Train Acc:  50.00%, Time: 0:00:12 *
Iter:     90, Train Loss:    1.4, Train Acc:  64.06%, Time: 0:00:17 *
Iter:    120, Train Loss:    1.1, Train Acc:  73.44%, Time: 0:00:22 *
Iter:    150, Train Loss:    1.0, Train Acc:  71.88%, Time: 0:00:23 
Iter:    180, Train Loss:   0.52, Train Acc:  89.06%, Time: 0:00:28 *
Epoch: 2
Iter:    210, Train Loss:   0.66, Train Acc:  84.38%, Time: 0:00:29 
Iter:    240, Train Loss:   0.84, Train Acc:  82.81%, Time: 0:00:30 
Iter:    270, Train Loss:   0.37, Train Acc:  90.62%, Time: 0:00:34 *
Iter:    300, Train Loss:   0.55, Train Acc:  84.38%, Time: 0:00:35 
Iter:    330, Train Loss:   0.42, Train Acc:  85.94%, Time: 0:00:36 
Iter:    360, Train Loss:    0.6, Train Acc:  81.25%, Time: 0:00:36 
Iter:    390, Train Loss:   0.48, Train Acc:  92.19%, Time: 0:00:43 *
Epoch: 3
Iter:    420, Train Loss:   0.25,

INFO:tensorflow:Restoring parameters from s3://corpus-2/model/cnn2/random/1/1


Testing...
Test Loss:   0.27, Test Acc:  91.52%
Precision, Recall and F1-Score...
                           precision    recall  f1-score   support

           Retrieve Value       0.89      0.95      0.92       152
                   Filter       0.79      0.91      0.85       136
    Compute Derived Value       0.92      0.85      0.88       117
            Find Extremum       0.95      0.89      0.92       164
                     Sort       0.94      0.94      0.94       124
          Determine Range       0.92      0.92      0.92       139
Characterize Distribution       0.91      0.88      0.90       130
           Find Anomalies       0.99      0.89      0.94       141
                  Cluster       0.95      0.95      0.95       140
                Correlate       0.91      0.95      0.93       161

                micro avg       0.92      0.92      0.92      1404
                macro avg       0.92      0.91      0.91      1404
             weighted avg       0.92      0.9

INFO:tensorflow:Restoring parameters from s3://corpus-2/model/cnn2/random/2/2


Testing...
Test Loss:   0.32, Test Acc:  89.53%
Precision, Recall and F1-Score...
                           precision    recall  f1-score   support

           Retrieve Value       0.93      0.94      0.94       109
                   Filter       0.95      0.87      0.91       156
    Compute Derived Value       0.89      0.88      0.88       141
            Find Extremum       0.89      0.95      0.92       190
                     Sort       0.96      0.88      0.92       138
          Determine Range       0.84      0.88      0.86       139
Characterize Distribution       0.95      0.83      0.89       148
           Find Anomalies       0.89      0.87      0.88       141
                  Cluster       0.80      0.93      0.86       114
                Correlate       0.87      0.92      0.90       128

                micro avg       0.90      0.90      0.90      1404
                macro avg       0.90      0.90      0.89      1404
             weighted avg       0.90      0.9

...

Testing...
Test Loss:    0.4, Test Acc:  87.88%
Precision, Recall and F1-Score...
                           precision    recall  f1-score   support

           Retrieve Value       0.83      0.93      0.88       136
                   Filter       0.78      0.89      0.83       139
    Compute Derived Value       0.92      0.84      0.88       140
            Find Extremum       0.86      0.95      0.90       159
                     Sort       0.88      0.98      0.93       129
          Determine Range       0.85      0.86      0.86       150
Characterize Distribution       0.90      0.89      0.90       133
           Find Anomalies       0.93      0.79      0.85       140
                  Cluster       0.95      0.81      0.88       151
                Correlate       0.94      0.85      0.89       126

                micro avg       0.88      0.88      0.88      1403
                macro avg       0.88      0.88      0.88      1403
             weighted avg       0.88      0.8

INFO:tensorflow:Restoring parameters from s3://corpus-2/model/cnn2/random/9/9


Testing...
Test Loss:   0.43, Test Acc:  87.67%
Precision, Recall and F1-Score...
                           precision    recall  f1-score   support

           Retrieve Value       0.84      0.95      0.89       128
                   Filter       0.83      0.87      0.85       161
    Compute Derived Value       0.89      0.87      0.88       185
            Find Extremum       0.88      0.88      0.88       145
                     Sort       0.94      0.93      0.93       112
          Determine Range       0.85      0.83      0.84       136
Characterize Distribution       0.87      0.87      0.87       127
           Find Anomalies       0.80      0.93      0.86       119
                  Cluster       0.93      0.89      0.91       125
                Correlate       0.95      0.80      0.87       165

                micro avg       0.88      0.88      0.88      1403
                macro avg       0.88      0.88      0.88      1403
             weighted avg       0.88      0.8

INFO:tensorflow:Restoring parameters from s3://corpus-2/model/cnn2/random/10/10


Testing...
Test Loss:   0.34, Test Acc:  89.95%
Precision, Recall and F1-Score...
                           precision    recall  f1-score   support

           Retrieve Value       0.88      0.96      0.92       109
                   Filter       0.93      0.79      0.86       158
    Compute Derived Value       0.87      0.92      0.89       144
            Find Extremum       0.92      0.92      0.92       182
                     Sort       0.98      0.88      0.93       128
          Determine Range       0.94      0.89      0.91       120
Characterize Distribution       0.96      0.88      0.92       126
           Find Anomalies       0.82      0.88      0.85       137
                  Cluster       0.87      0.93      0.90       135
                Correlate       0.87      0.96      0.91       164

                micro avg       0.90      0.90      0.90      1403
                macro avg       0.90      0.90      0.90      1403
             weighted avg       0.90      0.9

In [32]:
model_test_acc

[0.9152421642232824,
 0.8952991454689591,
 0.8931623935020208,
 0.8881766376671968,
 0.920227920397734,
 0.8959372766257522,
 0.9265858868318886,
 0.8788310771997877,
 0.8766927999508696,
 0.8995010680329859]

In [31]:
np.mean(model_test_acc),np.std(model_test_acc),np.std(model_test_acc,ddof=1),np.var(model_test_acc)

(0.8989656369900478,
 0.015982330737010338,
 0.016846855815690075,
 0.0002554348957871854)

In [101]:
def expert_split():
    train_data = [[] for i in range(20)]
    with open(trainDataPath, "r", encoding='utf-8') as fp:
        for line in fp.readlines():
            word = line.split()
            info = word[0]
            index = int(info.split(":")[4]) - 1
            label = int(word[0].split(":")[0])
            content = word[1:]
            train_data[index].append([content,label])
            
    for i in range(20):
        np.random.shuffle(train_data[i])
        train_data[i] = np.asarray(train_data[i])
        
    np.random.shuffle(train_data)
        
    return np.asarray(train_data)

In [102]:
train_data_expert = expert_split()

In [103]:
train_data_expert[0][:2]

array([[list(['which', 'player', "'s", 'transfer', '_fee', 'represents', 'an', 'anomaly', '?']),
        8],
       [list(['david', 'left', 'what', 'team', 'when', 'he', 'joined', 'the', 'premier', 'league', '?']),
        1]], dtype=object)

In [104]:
len(train_data_expert)

20

In [55]:
expert_train_acc = []
expert_test_acc = []
split_type = "expert"
fold_id = 0
for train_i, test_i in kf.split(x_train):
    fold_id += 1
    print("Fold: ", fold_id)
    expert_train_acc.append(model.train(x_train[train_i], y_train[train_i],split_type,fold_id))
    expert_test_acc.append(model.evaluate_model(x_train[test_i], y_train[test_i],split_type,fold_id,categories))

Fold:  1
Training and evaluating...
Epoch: 1
Iter:     30, Train Loss:    2.1, Train Acc:  37.50%, Time: 0:00:08 *
Iter:     60, Train Loss:    1.6, Train Acc:  62.50%, Time: 0:00:15 *
Iter:     90, Train Loss:    1.2, Train Acc:  59.38%, Time: 0:00:16 
Iter:    120, Train Loss:    1.1, Train Acc:  62.50%, Time: 0:00:16 
Iter:    150, Train Loss:   0.83, Train Acc:  81.25%, Time: 0:00:22 *
Iter:    180, Train Loss:   0.72, Train Acc:  85.94%, Time: 0:00:28 *
Epoch: 2
Iter:    210, Train Loss:   0.58, Train Acc:  84.38%, Time: 0:00:29 
Iter:    240, Train Loss:   0.61, Train Acc:  82.81%, Time: 0:00:30 
Iter:    270, Train Loss:   0.76, Train Acc:  82.81%, Time: 0:00:30 
Iter:    300, Train Loss:   0.47, Train Acc:  82.81%, Time: 0:00:31 
Iter:    330, Train Loss:   0.52, Train Acc:  84.38%, Time: 0:00:32 
Iter:    360, Train Loss:    0.4, Train Acc:  92.19%, Time: 0:00:39 *
Iter:    390, Train Loss:   0.46, Train Acc:  84.38%, Time: 0:00:39 
Epoch: 3
Iter:    420, Train Loss:   0.25, T

INFO:tensorflow:Restoring parameters from s3://corpus-2/model/cnn2/expert/1/1


Testing...
Test Loss:   0.35, Test Acc:  89.03%
Precision, Recall and F1-Score...
                           precision    recall  f1-score   support

           Retrieve Value       0.97      0.84      0.90       152
                   Filter       0.85      0.89      0.87       136
    Compute Derived Value       0.79      0.91      0.85       117
            Find Extremum       0.92      0.93      0.93       164
                     Sort       0.97      0.87      0.92       124
          Determine Range       0.94      0.91      0.93       139
Characterize Distribution       0.97      0.87      0.92       130
           Find Anomalies       0.70      0.97      0.81       141
                  Cluster       0.94      0.89      0.91       140
                Correlate       0.96      0.83      0.89       161

                micro avg       0.89      0.89      0.89      1404
                macro avg       0.90      0.89      0.89      1404
             weighted avg       0.90      0.8

INFO:tensorflow:Restoring parameters from s3://corpus-2/model/cnn2/expert/2/2


Testing...
Test Loss:    0.3, Test Acc:  91.03%
Precision, Recall and F1-Score...
                           precision    recall  f1-score   support

           Retrieve Value       0.95      0.93      0.94       109
                   Filter       0.91      0.88      0.90       156
    Compute Derived Value       0.92      0.91      0.92       141
            Find Extremum       0.88      0.96      0.92       190
                     Sort       0.92      0.95      0.94       138
          Determine Range       0.86      0.93      0.89       139
Characterize Distribution       0.98      0.83      0.90       148
           Find Anomalies       0.91      0.89      0.90       141
                  Cluster       0.89      0.89      0.89       114
                Correlate       0.91      0.92      0.91       128

                micro avg       0.91      0.91      0.91      1404
                macro avg       0.91      0.91      0.91      1404
             weighted avg       0.91      0.9

...

Testing...
Test Loss:   0.29, Test Acc:  91.45%
Precision, Recall and F1-Score...
                           precision    recall  f1-score   support

           Retrieve Value       0.96      0.93      0.95       136
                   Filter       0.83      0.90      0.87       139
    Compute Derived Value       0.89      0.95      0.92       140
            Find Extremum       0.93      0.94      0.93       159
                     Sort       0.94      0.95      0.94       129
          Determine Range       0.93      0.90      0.92       150
Characterize Distribution       0.91      0.89      0.90       133
           Find Anomalies       0.98      0.86      0.91       140
                  Cluster       0.94      0.91      0.92       151
                Correlate       0.85      0.93      0.89       126

                micro avg       0.91      0.91      0.91      1403
                macro avg       0.92      0.91      0.91      1403
             weighted avg       0.92      0.9

INFO:tensorflow:Restoring parameters from s3://corpus-2/model/cnn2/expert/9/9


Testing...
Test Loss:   0.34, Test Acc:  90.16%
Precision, Recall and F1-Score...
                           precision    recall  f1-score   support

           Retrieve Value       0.76      0.98      0.86       128
                   Filter       0.92      0.84      0.88       161
    Compute Derived Value       0.97      0.81      0.88       185
            Find Extremum       0.86      0.97      0.91       145
                     Sort       0.94      0.94      0.94       112
          Determine Range       0.89      0.85      0.87       136
Characterize Distribution       0.91      0.87      0.89       127
           Find Anomalies       0.88      0.95      0.91       119
                  Cluster       0.98      0.94      0.96       125
                Correlate       0.94      0.92      0.93       165

                micro avg       0.90      0.90      0.90      1403
                macro avg       0.91      0.91      0.90      1403
             weighted avg       0.91      0.9

INFO:tensorflow:Restoring parameters from s3://corpus-2/model/cnn2/expert/10/10


Testing...
Test Loss:   0.34, Test Acc:  89.45%
Precision, Recall and F1-Score...
                           precision    recall  f1-score   support

           Retrieve Value       0.82      0.99      0.90       109
                   Filter       0.89      0.84      0.86       158
    Compute Derived Value       0.94      0.85      0.89       144
            Find Extremum       0.86      0.96      0.91       182
                     Sort       0.98      0.88      0.93       128
          Determine Range       0.85      0.88      0.87       120
Characterize Distribution       0.98      0.86      0.92       126
           Find Anomalies       0.93      0.79      0.85       137
                  Cluster       0.93      0.93      0.93       135
                Correlate       0.82      0.96      0.89       164

                micro avg       0.89      0.89      0.89      1403
                macro avg       0.90      0.89      0.89      1403
             weighted avg       0.90      0.8

In [56]:
expert_test_acc

[0.8903133894643213,
 0.9102564092375275,
 0.8945868935680118,
 0.8896011385822568,
 0.9173789175487311,
 0.9052031355845613,
 0.9130434781759021,
 0.914468995945333,
 0.9016393437100069,
 0.8945117604282186]

In [57]:
np.mean(expert_test_acc),np.std(expert_test_acc),np.std(expert_test_acc,ddof=1),np.var(expert_test_acc)

(0.903100346224487,
 0.009908242579262691,
 0.010444204719977177,
 9.817327100951418e-05)

In [105]:
ex_train = []
ey_train = []
for ti in train_data_expert:
    x_train, y_train = process_file(ti[:,0], ti[:,1], word_to_id, num_classes, seq_length)
    ex_train.append(x_train)
    ey_train.append(y_train)

In [106]:
ex_train = np.asarray(ex_train)
ey_train = np.asarray(ey_train)

In [107]:
len(ex_train[0]),len(ex_train[1]),len(ex_train[0][0]),len(ex_train)

(636, 603, 41, 20)

In [108]:
def mergeData(data_x, data_y):
    merge_x = data_x[0]
    merge_y = data_y[0]
    for i in range(1,len(data_x)):
        merge_x = np.r_[merge_x,data_x[i]]
        merge_y = np.r_[merge_y,data_y[i]]
        
    return merge_x, merge_y

In [109]:
expert_train_acc = []
expert_test_acc = []
split_type = "expert"
fold_id = 0
for train_i, test_i in kf.split(ex_train):
    fold_id += 1
    print("Fold: ", fold_id)
    train_x, train_y = mergeData(ex_train[train_i],ey_train[train_i])
    test_x, test_y = mergeData(ex_train[test_i],ey_train[test_i])
    expert_train_acc.append(model.train(train_x, train_y,split_type,fold_id))
    expert_test_acc.append(model.evaluate_model(test_x, test_y,split_type,fold_id,categories))

Fold:  1
Training and evaluating...
Epoch: 1
Iter:     30, Train Loss:    2.1, Train Acc:  31.25%, Time: 0:00:09 *
Iter:     60, Train Loss:    1.7, Train Acc:  50.00%, Time: 0:00:14 *
Iter:     90, Train Loss:    1.4, Train Acc:  60.94%, Time: 0:00:20 *
Iter:    120, Train Loss:    1.2, Train Acc:  59.38%, Time: 0:00:21 
Iter:    150, Train Loss:    1.0, Train Acc:  73.44%, Time: 0:00:26 *
Iter:    180, Train Loss:   0.84, Train Acc:  73.44%, Time: 0:00:27 
Epoch: 2
Iter:    210, Train Loss:    0.7, Train Acc:  76.56%, Time: 0:00:33 *
Iter:    240, Train Loss:   0.57, Train Acc:  82.81%, Time: 0:00:39 *
Iter:    270, Train Loss:   0.42, Train Acc:  89.06%, Time: 0:00:45 *
Iter:    300, Train Loss:   0.41, Train Acc:  93.75%, Time: 0:00:52 *
Iter:    330, Train Loss:   0.33, Train Acc:  90.62%, Time: 0:00:53 
Iter:    360, Train Loss:   0.35, Train Acc:  85.94%, Time: 0:00:53 
Iter:    390, Train Loss:   0.17, Train Acc:  96.88%, Time: 0:00:59 *
Epoch: 3
Iter:    420, Train Loss:   0.1

INFO:tensorflow:Restoring parameters from s3://corpus-2/model/cnn2/expert/1/1


Testing...
Test Loss:    1.3, Test Acc:  63.36%
Precision, Recall and F1-Score...
                           precision    recall  f1-score   support

           Retrieve Value       0.66      0.70      0.68       124
                   Filter       0.53      0.75      0.62       119
    Compute Derived Value       0.61      0.49      0.54       131
            Find Extremum       0.80      0.76      0.78       123
                     Sort       0.88      0.66      0.75       121
          Determine Range       0.77      0.50      0.60       125
Characterize Distribution       0.69      0.89      0.78       127
           Find Anomalies       0.47      0.40      0.44       121
                  Cluster       0.62      0.49      0.55       123
                Correlate       0.48      0.70      0.57       125

                micro avg       0.63      0.63      0.63      1239
                macro avg       0.65      0.63      0.63      1239
             weighted avg       0.65      0.6

INFO:tensorflow:Restoring parameters from s3://corpus-2/model/cnn2/expert/2/2


Testing...
Test Loss:    1.5, Test Acc:  60.52%
Precision, Recall and F1-Score...
                           precision    recall  f1-score   support

           Retrieve Value       0.80      0.36      0.50       181
                   Filter       0.36      0.23      0.28       167
    Compute Derived Value       0.66      0.44      0.53       250
            Find Extremum       0.65      0.91      0.76       173
                     Sort       0.66      0.80      0.72       120
          Determine Range       0.50      0.74      0.59       178
Characterize Distribution       0.70      0.86      0.77        86
           Find Anomalies       0.48      0.88      0.62       140
                  Cluster       0.73      0.63      0.68       111
                Correlate       0.78      0.52      0.62       172

                micro avg       0.61      0.61      0.61      1578
                macro avg       0.63      0.64      0.61      1578
             weighted avg       0.63      0.6

...

Testing...
Test Loss:    1.3, Test Acc:  65.38%
Precision, Recall and F1-Score...
                           precision    recall  f1-score   support

           Retrieve Value       0.69      0.53      0.60       121
                   Filter       0.46      0.51      0.48       130
    Compute Derived Value       0.75      0.39      0.51       127
            Find Extremum       0.61      0.77      0.68       124
                     Sort       0.82      0.85      0.84       124
          Determine Range       0.79      0.72      0.76       120
Characterize Distribution       0.91      0.60      0.72       135
           Find Anomalies       0.69      0.48      0.57       143
                  Cluster       0.82      0.76      0.79       139
                Correlate       0.44      0.93      0.60       134

                micro avg       0.65      0.65      0.65      1297
                macro avg       0.70      0.65      0.65      1297
             weighted avg       0.70      0.6

INFO:tensorflow:Restoring parameters from s3://corpus-2/model/cnn2/expert/9/9


Testing...
Test Loss:    1.3, Test Acc:  65.44%
Precision, Recall and F1-Score...
                           precision    recall  f1-score   support

           Retrieve Value       0.43      0.61      0.50       120
                   Filter       0.44      0.71      0.54       126
    Compute Derived Value       0.47      0.46      0.47       124
            Find Extremum       0.79      0.82      0.80       129
                     Sort       0.85      0.85      0.85       124
          Determine Range       0.32      0.16      0.21       125
Characterize Distribution       0.79      0.83      0.81       127
           Find Anomalies       0.98      0.41      0.58       125
                  Cluster       0.88      0.75      0.81       125
                Correlate       0.84      0.92      0.88       128

                micro avg       0.65      0.65      0.65      1253
                macro avg       0.68      0.65      0.64      1253
             weighted avg       0.68      0.6

INFO:tensorflow:Restoring parameters from s3://corpus-2/model/cnn2/expert/10/10


Testing...
Test Loss:    1.2, Test Acc:  67.26%
Precision, Recall and F1-Score...
                           precision    recall  f1-score   support

           Retrieve Value       0.85      0.37      0.51       106
                   Filter       0.72      0.56      0.63       121
    Compute Derived Value       0.55      0.45      0.49       121
            Find Extremum       0.53      0.79      0.63       119
                     Sort       0.73      0.76      0.74       111
          Determine Range       0.69      0.84      0.75       130
Characterize Distribution       0.73      0.74      0.73       127
           Find Anomalies       0.67      0.71      0.69       123
                  Cluster       0.68      0.78      0.73       122
                Correlate       0.73      0.69      0.71       154

                micro avg       0.67      0.67      0.67      1234
                macro avg       0.69      0.67      0.66      1234
             weighted avg       0.69      0.6

In [110]:
expert_test_acc

[0.6335754637952963,
 0.6051964519595012,
 0.7520064208327671,
 0.6135812453329322,
 0.6810055863923866,
 0.7160963240836208,
 0.617182564486903,
 0.6538164999821412,
 0.6544293689899033,
 0.672609400710565]

In [111]:
np.mean(expert_test_acc),np.std(expert_test_acc),np.std(expert_test_acc,ddof=1),np.var(expert_test_acc)

(0.6599499326566016,
 0.04468911700992965,
 0.04710646545771709,
 0.0019971171791271836)

In [112]:
def bundle_split():
    train_data = [[] for i in range(920)]
    with open(trainDataPath, "r", encoding='utf-8') as fp:
        for line in fp.readlines():
            word = line.split()
            info = word[0].split(":")
            index = int(info[1]) - 1
            label = int(info[0])
            content = word[1:]
            train_data[index].append([content,label])
            
    for i in range(920):
        np.random.shuffle(train_data[i])
        train_data[i] = np.asarray(train_data[i])
        
    np.random.shuffle(train_data)   
    
    return train_data

train_data_bundle = bundle_split()

In [113]:
train_data_bundle[0][:5,1]

array([7, 7, 7, 7, 7], dtype=object)

In [115]:
def mergeData(data_x, data_y):
    merge_x = data_x[0]
    merge_y = data_y[0]
    for i in range(1,len(data_x)):
        merge_x = np.r_[merge_x,data_x[i]]
        merge_y = np.r_[merge_y,data_y[i]]
        
    return merge_x, merge_y


def train_split_data(train_data, split_type):
    
    print(split_type)
    
    tx = []
    ty = []
    for ti in train_data:
        x_train, y_train = process_file(ti[:,0], ti[:,1], word_to_id, num_classes, seq_length)
        tx.append(x_train)
        ty.append(y_train)

    tx = np.asarray(tx)
    ty = np.asarray(ty)
    
    print(len(tx),len(tx[0]),len(tx[1]),len(tx[0][0]))
    
    train_acc = []
    test_acc = []
    fold_id = 0
    
    for train_i, test_i in kf.split(tx):
        fold_id += 1
        print("Fold: ", fold_id)
        train_x, train_y = mergeData(tx[train_i],ty[train_i])
        test_x, test_y = mergeData(tx[test_i],ty[test_i])
        train_acc.append(model.train(train_x, train_y,split_type,fold_id))
        test_acc.append(model.evaluate_model(test_x, test_y,split_type,fold_id,categories))
        
    print(test_acc)
    print(np.mean(test_acc),np.std(test_acc),np.std(test_acc,ddof=1),np.var(test_acc))
    
    return test_acc

In [116]:
bundle_test_acc = train_split_data(train_data_bundle, "bundle")

bundle
920 15 13 41
Fold:  1
Training and evaluating...
Epoch: 1
Iter:     30, Train Loss:    2.0, Train Acc:  39.06%, Time: 0:00:08 *
Iter:     60, Train Loss:    1.6, Train Acc:  54.69%, Time: 0:00:13 *
Iter:     90, Train Loss:    1.3, Train Acc:  65.62%, Time: 0:00:18 *
Iter:    120, Train Loss:    1.1, Train Acc:  71.88%, Time: 0:00:24 *
Iter:    150, Train Loss:   0.96, Train Acc:  78.12%, Time: 0:00:29 *
Iter:    180, Train Loss:   0.92, Train Acc:  73.44%, Time: 0:00:30 
Epoch: 2
Iter:    210, Train Loss:   0.72, Train Acc:  84.38%, Time: 0:00:35 *
Iter:    240, Train Loss:    0.5, Train Acc:  84.38%, Time: 0:00:36 
Iter:    270, Train Loss:   0.61, Train Acc:  85.94%, Time: 0:00:40 *
Iter:    300, Train Loss:   0.38, Train Acc:  90.62%, Time: 0:00:45 *
Iter:    330, Train Loss:   0.44, Train Acc:  87.50%, Time: 0:00:46 
Iter:    360, Train Loss:   0.31, Train Acc:  95.31%, Time: 0:00:50 *
Iter:    390, Train Loss:   0.31, Train Acc:  90.62%, Time: 0:00:51 
Epoch: 3
Iter:    42

INFO:tensorflow:Restoring parameters from s3://corpus-2/model/cnn2/bundle/1/1


Testing...
Test Loss:   0.88, Test Acc:  75.85%
Precision, Recall and F1-Score...
                           precision    recall  f1-score   support

           Retrieve Value       0.84      0.88      0.86       163
                   Filter       0.84      0.69      0.76       133
    Compute Derived Value       0.80      0.54      0.65       197
            Find Extremum       0.61      0.99      0.75        90
                     Sort       0.86      0.90      0.88       155
          Determine Range       0.79      0.83      0.81       150
Characterize Distribution       0.66      0.77      0.71       119
           Find Anomalies       0.66      0.73      0.69       131
                  Cluster       0.77      0.74      0.75       136
                Correlate       0.75      0.66      0.70       175

                micro avg       0.76      0.76      0.76      1449
                macro avg       0.76      0.77      0.76      1449
             weighted avg       0.77      0.7

INFO:tensorflow:Restoring parameters from s3://corpus-2/model/cnn2/bundle/2/2


Testing...
Test Loss:    1.4, Test Acc:  63.12%
Precision, Recall and F1-Score...
                           precision    recall  f1-score   support

           Retrieve Value       0.64      0.62      0.63        90
                   Filter       0.50      0.45      0.47       162
    Compute Derived Value       0.69      0.69      0.69       189
            Find Extremum       0.53      0.76      0.63        92
                     Sort       0.78      0.75      0.77        76
          Determine Range       0.65      0.51      0.57       240
Characterize Distribution       0.76      0.71      0.73       137
           Find Anomalies       0.57      0.42      0.48       109
                  Cluster       0.86      0.66      0.75       130
                Correlate       0.52      0.88      0.65       139

                micro avg       0.63      0.63      0.63      1364
                macro avg       0.65      0.65      0.64      1364
             weighted avg       0.65      0.6

...

Testing...
Test Loss:   0.91, Test Acc:  75.34%
Precision, Recall and F1-Score...
                           precision    recall  f1-score   support

           Retrieve Value       0.71      0.83      0.77       168
                   Filter       0.72      0.64      0.68       129
    Compute Derived Value       0.60      0.65      0.62       164
            Find Extremum       0.81      0.87      0.84       149
                     Sort       0.91      0.93      0.92       147
          Determine Range       0.87      0.77      0.82       147
Characterize Distribution       0.79      0.74      0.76       156
           Find Anomalies       0.48      0.77      0.59        65
                  Cluster       0.76      0.70      0.73       106
                Correlate       0.88      0.63      0.74       172

                micro avg       0.75      0.75      0.75      1403
                macro avg       0.75      0.75      0.75      1403
             weighted avg       0.77      0.7

INFO:tensorflow:Restoring parameters from s3://corpus-2/model/cnn2/bundle/9/9


Testing...
Test Loss:    1.0, Test Acc:  71.50%
Precision, Recall and F1-Score...
                           precision    recall  f1-score   support

           Retrieve Value       0.83      0.73      0.78       228
                   Filter       0.53      0.74      0.62       149
    Compute Derived Value       0.57      0.62      0.59       129
            Find Extremum       0.89      0.76      0.82       175
                     Sort       0.80      0.96      0.87        98
          Determine Range       0.69      0.51      0.58        89
Characterize Distribution       0.94      0.85      0.90        96
           Find Anomalies       0.74      0.66      0.70       165
                  Cluster       0.84      0.70      0.76       139
                Correlate       0.45      0.62      0.52       104

                micro avg       0.72      0.72      0.72      1372
                macro avg       0.73      0.71      0.71      1372
             weighted avg       0.74      0.7

INFO:tensorflow:Restoring parameters from s3://corpus-2/model/cnn2/bundle/10/10


Testing...
Test Loss:    1.3, Test Acc:  66.57%
Precision, Recall and F1-Score...
                           precision    recall  f1-score   support

           Retrieve Value       0.72      0.67      0.70       189
                   Filter       0.48      0.36      0.42       162
    Compute Derived Value       0.78      0.54      0.63       198
            Find Extremum       0.78      0.82      0.80       224
                     Sort       0.78      0.58      0.66        90
          Determine Range       0.76      0.72      0.74       152
Characterize Distribution       0.53      0.78      0.63       120
           Find Anomalies       0.44      0.64      0.52        80
                  Cluster       0.81      0.77      0.79        91
                Correlate       0.57      0.86      0.69        91

                micro avg       0.67      0.67      0.67      1397
                macro avg       0.67      0.67      0.66      1397
             weighted avg       0.68      0.6

In [117]:
def table_split():
    train_data = [[] for i in range(37)]
    with open(trainDataPath, "r", encoding='utf-8') as fp:
        for line in fp.readlines():
            word = line.split()
            info = word[0].split(":")
            index = int(info[3]) - 1
            label = int(info[0])
            content = word[1:]
            train_data[index].append([content,label])
            
    for i in range(37):
        np.random.shuffle(train_data[i])
        train_data[i] = np.asarray(train_data[i])
        
    np.random.shuffle(train_data)   
    
    return train_data

train_data_table = table_split()

In [118]:
table_test_acc = train_split_data(train_data_table, "table")

table
37 610 296 41
Fold:  1
Training and evaluating...
Epoch: 1
Iter:     30, Train Loss:    2.0, Train Acc:  40.62%, Time: 0:00:08 *
Iter:     60, Train Loss:    1.7, Train Acc:  43.75%, Time: 0:00:14 *
Iter:     90, Train Loss:    1.3, Train Acc:  71.88%, Time: 0:00:20 *
Iter:    120, Train Loss:    1.1, Train Acc:  67.19%, Time: 0:00:21 
Iter:    150, Train Loss:    1.1, Train Acc:  67.19%, Time: 0:00:22 
Iter:    180, Train Loss:   0.82, Train Acc:  82.81%, Time: 0:00:28 *
Epoch: 2
Iter:    210, Train Loss:   0.49, Train Acc:  84.38%, Time: 0:00:33 *
Iter:    240, Train Loss:    0.7, Train Acc:  76.56%, Time: 0:00:34 
Iter:    270, Train Loss:   0.45, Train Acc:  90.62%, Time: 0:00:38 *
Iter:    300, Train Loss:   0.53, Train Acc:  84.38%, Time: 0:00:39 
Iter:    330, Train Loss:   0.31, Train Acc:  92.19%, Time: 0:00:43 *
Iter:    360, Train Loss:   0.42, Train Acc:  87.50%, Time: 0:00:44 
Iter:    390, Train Loss:   0.52, Train Acc:  87.50%, Time: 0:00:45 
Epoch: 3
Iter:    420,

INFO:tensorflow:Restoring parameters from s3://corpus-2/model/cnn2/table/1/1


Testing...
Test Loss:    1.5, Test Acc:  64.30%
Precision, Recall and F1-Score...
                           precision    recall  f1-score   support

           Retrieve Value       0.67      0.29      0.40       132
                   Filter       0.51      0.33      0.40       128
    Compute Derived Value       0.38      0.59      0.46       138
            Find Extremum       0.73      0.81      0.77       200
                     Sort       0.66      0.76      0.71       107
          Determine Range       0.88      0.48      0.62       125
Characterize Distribution       0.77      0.93      0.84       124
           Find Anomalies       0.87      0.62      0.72       115
                  Cluster       0.84      0.65      0.73       118
                Correlate       0.52      0.93      0.66       121

                micro avg       0.64      0.64      0.64      1308
                macro avg       0.68      0.64      0.63      1308
             weighted avg       0.68      0.6

INFO:tensorflow:Restoring parameters from s3://corpus-2/model/cnn2/table/2/2


Testing...
Test Loss:    1.3, Test Acc:  63.42%
Precision, Recall and F1-Score...
                           precision    recall  f1-score   support

           Retrieve Value       0.68      0.32      0.43       157
                   Filter       0.36      0.56      0.44       144
    Compute Derived Value       0.61      0.36      0.45       158
            Find Extremum       0.78      0.63      0.70       202
                     Sort       0.87      0.91      0.89       181
          Determine Range       0.71      0.58      0.63       158
Characterize Distribution       0.86      0.67      0.75       158
           Find Anomalies       0.44      0.79      0.57       157
                  Cluster       0.70      0.84      0.76       159
                Correlate       0.61      0.63      0.62       177

                micro avg       0.63      0.63      0.63      1651
                macro avg       0.66      0.63      0.63      1651
             weighted avg       0.67      0.6

...

Testing...
Test Loss:    1.5, Test Acc:  60.58%
Precision, Recall and F1-Score...
                           precision    recall  f1-score   support

           Retrieve Value       0.48      0.23      0.31        96
                   Filter       0.40      0.60      0.48       120
    Compute Derived Value       0.65      0.37      0.47       128
            Find Extremum       0.60      0.89      0.72       128
                     Sort       0.70      0.74      0.72        86
          Determine Range       0.50      0.23      0.32        91
Characterize Distribution       0.53      0.63      0.57       110
           Find Anomalies       0.90      0.53      0.67       105
                  Cluster       0.67      0.90      0.76       105
                Correlate       0.76      0.86      0.81       109

                micro avg       0.61      0.61      0.61      1078
                macro avg       0.62      0.60      0.58      1078
             weighted avg       0.62      0.6

INFO:tensorflow:Restoring parameters from s3://corpus-2/model/cnn2/table/9/9


Testing...
Test Loss:    1.1, Test Acc:  69.85%
Precision, Recall and F1-Score...
                           precision    recall  f1-score   support

           Retrieve Value       0.55      0.83      0.66        60
                   Filter       0.54      0.37      0.44        59
    Compute Derived Value       0.53      0.77      0.62        64
            Find Extremum       0.88      0.81      0.85       128
                     Sort       0.74      0.78      0.76        63
          Determine Range       0.66      0.48      0.56        85
Characterize Distribution       0.93      0.53      0.67        80
           Find Anomalies       0.48      0.92      0.63        60
                  Cluster       0.83      0.74      0.78        88
                Correlate       0.85      0.72      0.78       139

                micro avg       0.70      0.70      0.70       826
                macro avg       0.70      0.69      0.68       826
             weighted avg       0.74      0.7

INFO:tensorflow:Restoring parameters from s3://corpus-2/model/cnn2/table/10/10


Testing...
Test Loss:    1.3, Test Acc:  64.53%
Precision, Recall and F1-Score...
                           precision    recall  f1-score   support

           Retrieve Value       0.46      0.69      0.55        68
                   Filter       0.52      0.64      0.57        78
    Compute Derived Value       0.54      0.51      0.52        85
            Find Extremum       0.83      0.91      0.87        65
                     Sort       0.89      0.55      0.68        60
          Determine Range       0.84      0.95      0.89        62
Characterize Distribution       0.59      0.75      0.66        63
           Find Anomalies       0.54      0.40      0.46        62
                  Cluster       0.79      0.52      0.63        60
                Correlate       0.78      0.57      0.66        68

                micro avg       0.65      0.65      0.65       671
                macro avg       0.68      0.65      0.65       671
             weighted avg       0.67      0.6

In [ ]:
expert_train_acc = []
expert_test_acc = []
split_type = "expert"
fold_id = 0
for train_i, test_i in kf.split(ex_train):
    fold_id += 1
    print("Fold: ", fold_id)
    train_x, train_y = mergeData(ex_train[train_i],ey_train[train_i])
    test_x, test_y = mergeData(ex_train[test_i],ey_train[test_i])
    expert_train_acc.append(model.train(train_x, train_y,split_type,fold_id))
    expert_test_acc.append(model.evaluate_model(test_x, test_y,split_type,fold_id,categories))


bundle_train_acc = []
bundle_test_acc = []
split_type = "bundle"
fold_id = 0
for train_i, test_i in kf.split(bx_train):
    fold_id += 1
    print("Fold: ", fold_id)
    expert_train_acc.append(model.train(x_train[train_i], y_train[train_i],split_type,fold_id))
    expert_test_acc.append(model.evaluate_model(x_train[test_i], y_train[test_i],split_type,fold_id,categories))

In [26]:
np.mean(model_test_acc),np.std(model_test_acc),np.std(model_test_acc,ddof=1),np.var(model_test_acc)

(0.6393368500847791,
 0.05406597768875925,
 0.05699054447344181,
 0.0029231299434414135)